In [ ]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments
from datasets import load_dataset, Features, Value

#加载模型
max_seq_length = 2048
dtype = None
load_in_4bit = False
#本地模型目录
cache_dir = '/root/.cache/modelscope/hub/models'
model_name = f'{cache_dir}/Qwen/Qwen2.5-7B-Instruct'
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    model_type='qwen2',
    cache_dir=cache_dir
)

==((====))==  Unsloth 2025.3.15: Fast Qwen2 patching. Transformers: 4.49.0.
   \\   /|    NVIDIA A30. Num GPUs = 4. Max memory: 23.486 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [33]:
# 训练数据集
train_dataset_path = './datasets/alpaca_train_dataset.jsonl'

EOS_TOKEN = tokenizer.eos_token # 必须添加 EOS_TOKEN

_train_features = Features({
    'instruction': Value('string'),
    'input': Value('string'),
    'output': Value('string')
})

train_dataset = load_dataset(
    "json", 
    data_files=train_dataset_path, 
    split="train", 
    features=_train_features,
)
train_dataset

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 5000
})

In [ ]:
def formatting_alpaca_prompts_func(example):
    alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
    ### Instruction:
    {}
    ### Input:
    {}
    ### Output:
    {}"""

    instruction = example['instruction']
    input_text = example['input']
    output_text = example['output']
    
    alpaca_prompt = alpaca_prompt.format(instruction, input_text, output_text)
    
    return {"text": alpaca_prompt}

train_dataset = train_dataset.map(formatting_alpaca_prompts_func, remove_columns=['instruction','input','output'])
train_dataset[0]

In [36]:
#设置训练参数
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                  "gate_proj", "up_proj", "down_proj",],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing=True,
    random_state=322,
    max_seq_length=max_seq_length,
    use_rslora=False,
    loftq_config=None,
)

Unsloth 2025.3.15 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
checkpoint_save_path='/data/healthai/checkpoints'
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        max_steps=60,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        output_dir=checkpoint_save_path, # checkpoints 存放目录
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=322,
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=64):   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
#开始训练
trainer.train()

#保存微调模型
model.save_pretrained("Qwen-2.5-7B-Instruct-Lora")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 2
   \\   /|    Num examples = 5,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 40,370,176/7,655,986,688 (0.53% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,25.013800
2,2.811800
3,-43.717600
4,3.152000
5,0.356300
6,1.824100
7,0.334900
8,0.000000
9,0.000000
10,7.884300
